In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/AuNP_model

# Photo Preprocessing

In [ ]:
# import
import os
import numpy as np
import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers
# from tensorflow.keras import models

from cv2 import imread, IMREAD_GRAYSCALE
from pandas import read_csv
# import matplotlib.pyplot as plt

In [ ]:
# Seed
SEED = 1234
tf.random.set_seed(SEED)

In [ ]:
PATH = "training_data"
file_dir = os.listdir(PATH)
file_dir.sort()
X_train = np.array([])

tmp = []

prep = read_csv("auto_label.csv")
N = 0

for i in range(len(file_dir)):
    if prep['label'][i]:
        img = imread(PATH + "/" + file_dir[i], IMREAD_GRAYSCALE)
        img = np.expand_dims(img[:], axis = 2)
        tmp.append(img)
        N += 1

X_train = np.array(tmp) / 255

# Prepare y_train
data = read_csv("color_label.csv")
y_train = data["label"]
y_train = np.expand_dims(y_train[:N], axis = 1)

In [ ]:
print(N)

225


In [ ]:
# Shuffle
from sklearn.model_selection import train_test_split

val_split = 0.1
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=val_split, stratify=y_train)

# X_train = np.concatenate((X_t, X_v))
# y_train = np.concatenate((y_t, y_v))

In [ ]:
# Data augmentation and splitting data
from keras_preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
                    width_shift_range=0.1, height_shift_range=0.1, zoom_range=0.3,
                    validation_split=0.1)

training = train_datagen.flow(
                    X_train, y_train, batch_size=32, subset='training')

validation = train_datagen.flow(
                    X_train, y_train, batch_size=16, subset='validation')

In [ ]:
from sklearn.model_selection import train_test_split

# Linear model
# X_train_linear, X_val_linear, y_train_linear, y_val_linear = train_test_split(X_train, y_train, test_size=0.2, random_state=1)
training_linearX = np.reshape(training.x, (training.x.shape[0], -1))
validation_linearX = np.reshape(validation.x, (validation.x.shape[0], -1))

model_linear = tf.keras.models.Sequential()
model_linear.add(tf.keras.layers.Dense(1))

model_linear.compile(optimizer='sgd',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy(),])

history_linear = model_linear.fit(training_linearX, training.y, batch_size=16, epochs=10, validation_data=(validation_linearX, validation.y))

Epoch 1/10
12/12 [==============================] - 1s 24ms/step - loss: 4.4499 - binary_accuracy: 0.6538 - val_loss: 3.8123 - val_binary_accuracy: 0.7500
Epoch 2/10
12/12 [==============================] - 0s 7ms/step - loss: 4.3569 - binary_accuracy: 0.7143 - val_loss: 3.8123 - val_binary_accuracy: 0.7500
Epoch 3/10
12/12 [==============================] - 0s 7ms/step - loss: 4.3569 - binary_accuracy: 0.7143 - val_loss: 3.8123 - val_binary_accuracy: 0.7500
Epoch 4/10
12/12 [==============================] - 0s 8ms/step - loss: 4.3569 - binary_accuracy: 0.7143 - val_loss: 3.8123 - val_binary_accuracy: 0.7500
Epoch 5/10
12/12 [==============================] - 0s 8ms/step - loss: 4.3569 - binary_accuracy: 0.7143 - val_loss: 3.8123 - val_binary_accuracy: 0.7500
Epoch 6/10
12/12 [==============================] - 0s 8ms/step - loss: 4.3569 - binary_accuracy: 0.7143 - val_loss: 3.8123 - val_binary_accuracy: 0.7500
Epoch 7/10
12/12 [==============================] - 0s 7ms/step - loss: 4.3

In [ ]:
X_test_linear = np.reshape(X_test, (X_test.shape[0], -1))

predict_linear = model_linear.predict(X_test_linear) #; print(predict_linear)

predict_linear = (predict_linear > 0.5).astype('int32').flatten()

from sklearn.metrics import f1_score
score_linear = f1_score(y_test, predict_linear); print(score_linear)

# mse_linear = tf.keras.losses.mean_squared_error(y_test, predict_linear).numpy(); print(mse_linear)
# mae_linear = tf.keras.losses.mean_absolute_error(y_test, predict_linear).numpy(); print(mae_linear)

1/1 [==============================] - 0s 108ms/step
0.8205128205128205


In [ ]:
# build model
model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(336, 252, 1)))
# model.add(tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'))
model.add(tf.keras.layers.Conv2D(64, 2, padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(2, strides=2))
model.add(tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(2, padding='same'))
# model.add(Activation('relu'))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy(), ])

model.summary()

history = model.fit(training, batch_size=8, epochs=20, validation_data=validation)

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 336, 252, 64)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 168, 126, 64)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 168, 126, 64)      36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 84, 63, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 338688)            0         
                                                                 
 dense_6 (Dense)             (None, 64)               

In [ ]:
predict = model.predict(X_test)

predict = (predict > 0.5).astype('int32').flatten()

from sklearn.metrics import f1_score
score = f1_score(y_test, predict); print(score)
# mse = tf.keras.losses.mean_squared_error(y_test, predict).numpy(); print(mse)
# mae = tf.keras.losses.mean_absolute_error(y_test, predict).numpy(); print(mae)

1/1 [==============================] - 1s 762ms/step
0.8205128205128205


# Save Model

In [ ]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)